In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import uuid
import scipy.spatial.distance as dist

from abc import ABC, abstractmethod


In [2]:
class Node:

    node_list = []

    def __init__(self, coordinates: tuple, logo: str) -> None:
        self.id = uuid.uuid4()
        self.coordinates = coordinates
        self.logo = logo
        self.element_type = None
        self.node_list.append(self)

    def __repr__(self) -> str:
        return f"Node({self.id}, {self.coordinates}, {self.logo})"

    def place_on_grid(self, dataframe):
        row,col = self.coordinates
        dataframe[row][col] = self.logo
        return dataframe
    
    def euclidean_distance(self, node):
        return dist.euclidean(self.coordinates, node.coordinates)

class Buidling(Node):

    def __init__(self, coordinates: tuple, logo: str) -> None:
        super().__init__(coordinates, logo)
        self.element_type = "building"

class Pavement(Node):

    def __init__(self, coordinates: tuple, logo: str) -> None:
        super().__init__(coordinates, logo)
        self.element_type = "pavement"



In [3]:
# Traffic Graph Class
class TrafficGraph(nx.Graph):

    def __init__(self, element_type: str) -> None:
        super().__init__()
        self.element_type = element_type
        self.node_register = {node.id: node for node in Node.node_list if node.element_type == self.element_type}

    def add_node(self, node: Node) -> None:
        super().add_node(node.id)
        self.node_register[node.id] = node


### Create Nodes

In [4]:


# create buidlings
groceries = Buidling((0,0), 'G')
school = Buidling((0,9), 'S')
home = Buidling((9,0), 'H')

# create pavements froom groceries to school 
for i in range(1, 9):
    Pavement((0,i), 'P')

# create pavements from grocieres to home
for i in range(1, 9):
    Pavement((i,0), 'P')




In [5]:
def initalize_grid(symbol: str='*') -> pd.DataFrame:
    grid = pd.DataFrame(np.chararray((10,10), itemsize=2))
    grid[:] = symbol
    for node in Node.node_list:
        grid = node.place_on_grid(grid)
    return grid

# create a grid
grid = initalize_grid()
grid

,0,1,2,3,4,5,6,7,8,9
0,G,P,P,P,P,P,P,P,P,H
1,P,*,*,*,*,*,*,*,*,*
2,P,*,*,*,*,*,*,*,*,*
3,P,*,*,*,*,*,*,*,*,*
4,P,*,*,*,*,*,*,*,*,*
5,P,*,*,*,*,*,*,*,*,*
6,P,*,*,*,*,*,*,*,*,*
7,P,*,*,*,*,*,*,*,*,*
8,P,*,*,*,*,*,*,*,*,*
9,S,*,*,*,*,*,*,*,*,*


In [6]:
pavements = TrafficGraph("pavement")
buildings = TrafficGraph("building")

